In [1]:
import xarray as xr
import numpy as np
import glob
import intake
import dask
from numcodecs import Blosc
import os

In [7]:
def create_directory(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f"Directory '{directory_path}' created.")
    else:
        print(f"Directory '{directory_path}' already exists.")

In [2]:
cat = intake.open_catalog("/home/a/a270088/PYTHON/nextgems/catalog/FESOM/IFS_28-FESOM_25-cycle3.yaml")
print(list(cat))

['2D_1h_native', '2D_daily_native', '2D_monthly_native', '3D_3h_native', '3D_daily_native', 'elem_grid', 'node_grid']


In [3]:
# run = cat.FESOM['IFS_4.4-FESOM_5-cycle3']
print(list(cat))

['2D_1h_native', '2D_daily_native', '2D_monthly_native', '3D_3h_native', '3D_daily_native', 'elem_grid', 'node_grid']


In [3]:
from dask.distributed import Client
import dask
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})
dask.config.set({'temporary_directory': '/scratch/a/a270088/dask/'})
# client = Client(n_workers=5, threads_per_worker=1, memory_limit='50GB')
client = Client(n_workers=20, threads_per_worker=1, memory_limit='10GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,Workers: 20
Total threads: 20,Total memory: 186.26 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41321,Workers: 20
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,Total threads: 20
Started: Just now,Total memory: 186.26 GiB
Comm: tcp://127.0.0.1:41879,Total threads: 1
Dashboard: /user/a270088/levante-spawner-preset//proxy/43345/status,Memory: 9.31 GiB
Nanny: tcp://127.0.0.1:37031,


## Compute daily means from hourly

In [5]:
# with dask.config.set(array_slicing_split_large_chunks=True):
data = cat['2D_1h_native'].to_dask()

In [6]:
list(data.variables)

['time',
 'MLD1',
 'MLD2',
 'a_ice',
 'atmice_x',
 'atmice_y',
 'atmoce_x',
 'atmoce_y',
 'evap',
 'fh',
 'fw',
 'm_ice',
 'm_snow',
 'prec',
 'ssh',
 'sss',
 'sst',
 'uice',
 'vice']

In [8]:
variables = ['MLD1',
 'MLD2',
 'a_ice',
 'atmice_x',
 'atmice_y',
 'atmoce_x',
 'atmoce_y',
 'evap',
 'fh',
 'fw',
 'm_ice',
 'm_snow',
 'prec',
 'ssh',
 'sss',
 'sst',
 'uice',
 'vice']

In [10]:
for year in range(2020, 2025):
    print(year)
    # for vari in ['u', 'v', 'w']:
    for vari in variables:
        print(f"variable: {vari}, year: {year}")
        vari_mm = data[vari].sel(time=str(year)).resample(time='1D').mean(dim='time')
        odir = f'/work/bm1344/AWI/Cycle3/IFS_28-FESOM_25-cycle3/daily_means/{vari}/'
        create_directory(odir)
        vari_mm.to_netcdf(f'{odir}/{vari}_{str(year)}.nc')

## Fake average runoff to have all daily means in one place

In [11]:
data = cat['2D_daily_native'].to_dask()

In [12]:
list(data.variables)

['time', 'runoff']

Here we just basically copy runoff, so that we have consistent time stamps and everythin in one directory

In [13]:
for year in range(2020, 2025):
    print(year)
    # for vari in ['u', 'v', 'w']:
    for vari in ['runoff']:
        print(f"variable: {vari}, year: {year}")
        vari_mm = data[vari].sel(time=str(year)).resample(time='1D').mean(dim='time')
        odir = f'/work/bm1344/AWI/Cycle3/IFS_28-FESOM_25-cycle3/daily_means/{vari}/'
        create_directory(odir)
        vari_mm.to_netcdf(f'{odir}/{vari}_{str(year)}.nc')

2020
variable: runoff, year: 2020
Directory '/work/bm1344/AWI/Cycle3/IFS_28-FESOM_25-cycle3/daily_means/runoff/' created.
2021
variable: runoff, year: 2021
Directory '/work/bm1344/AWI/Cycle3/IFS_28-FESOM_25-cycle3/daily_means/runoff/' already exists.
2022
variable: runoff, year: 2022
Directory '/work/bm1344/AWI/Cycle3/IFS_28-FESOM_25-cycle3/daily_means/runoff/' already exists.
2023
variable: runoff, year: 2023
Directory '/work/bm1344/AWI/Cycle3/IFS_28-FESOM_25-cycle3/daily_means/runoff/' already exists.
2024
variable: runoff, year: 2024
Directory '/work/bm1344/AWI/Cycle3/IFS_28-FESOM_25-cycle3/daily_means/runoff/' already exists.


## Convert daily 3D to monthly

In [4]:
data = cat['3D_daily_native'].to_dask()

In [5]:
list(data.variables)

['nz', 'time', 'Av', 'Kv', 'nz1', 'salt', 'temp', 'u', 'v', 'w']

In [ ]:
for year in range(2020, 2025):
    print(year)
    # for vari in ['u', 'v', 'w']:
    for vari in ['salt', 'temp', 'u', 'v', 'w']:
        print(f"variable: {vari}, year: {year}")
        vari_mm = data[vari].sel(time=str(year)).resample(time='1M').mean(dim='time')
        odir = f'/work/bm1344/AWI/Cycle3/IFS_28-FESOM_25-cycle3/monthly_means/{vari}/'
        create_directory(odir)
        vari_mm.to_netcdf(f'{odir}/{vari}_{str(year)}.nc')